# Data Acquisition Notebook

## Students

* Team: `18`
* Students: `Quentin Bacuet, Ali Alami-Idrissi, Keshav Singh, Leandro Kieliger`
* Dataset: `US-Senators`

## Description

This notebook contains the code necessary to collect data from the ProPublica Congress API. In addition, data undergoes some basic processing steps before being stored in .pickle format.

## Required librairies

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import os
import re

data_folder = 'data/'

## Loading the API key

The API key should be stored in a text file alongside the project. To obtain the API key, visit https://projects.propublica.org/api-docs/congress-api/

In [3]:
API_KEY = ""

with open('api_key.txt') as f:
    API_KEY = f.read()

HEADERS = {"X-Api-Key": API_KEY}

## Creating required folders

In [3]:
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_folder(data_folder)
create_folder(data_folder + 'committees')
create_folder(data_folder + 'senate_members')
create_folder(data_folder + 'votes')
create_folder(data_folder + 'lobby')
create_folder(data_folder + 'bills')

## Fetching bills

Bills can be fetched by querying the following url: 
https://api.propublica.org/congress/v1/{congress}/senate/bills/introduced.json?offset={offset}

* The `congress` parameter is used to specify the congress for which we want to query bills. A congress is a meeting of the American legislative branch which lasts two years. It is composed of two chambers, the senate and the house of representatives. Senators are elected for a period of 6 years.

* The `offset` parameter is used to navigate through the results since only 20 results are returned per request.

In [ ]:
bill_str = "https://api.propublica.org/congress/v1/{congress}/senate/bills/introduced.json?offset={offset}"

bill_list = []
more_data_available = True
request_offset = 0

while more_data_available :
    res = requests.get(url = bill_str.format(congress=115, offset = request_offset), headers = HEADERS)
    if('results' in res.json()):
        
        # Print progress status
        print(request_offset, end='\r')
        
        # Verify whether there might be additional result to query
        more_data_available = int(res.json()['results'][0]['num_results']) > 0

        if more_data_available:
            # Extract results from JSON response
            bills = res.json()['results'][0]['bills']
            bill_list.append(pd.io.json.json_normalize(bills, record_prefix=True))
    else:
        print(str(request_offset) + ' - Error: ' + res.json()['error'])
    request_offset += 20
    

In [ ]:
# Save all bills
df = pd.concat(bill_list,sort=True)
df.to_csv(data_folder + "bills/bills.csv")
df.to_pickle(data_folder + "bills/bills.pickle")

# Save only bills that are considered active. That is, bills that are being considered by at least one committee
active_bills = df[(df['active']==1)]
active_bills.to_csv(data_folder + "bills/active_bills.csv")
active_bills.to_pickle(data_folder + "bills/active_bills.pickle")

## Fetching co-sponsors for each bill

Each bill has a sponsor ID, this is the ID of the senator that initially pushes the bill for consideration. However, a bill can be co-sponsored by multiple senators. To find those co-sponsors we need ot perform additional requests.

In [ ]:
def extract_cosponsors(cosponsor_list):
    """
    Extract the cosponsor IDs from a list of JSONs
    """
    
    return [x['cosponsor_id'] for x in cosponsor_list]

def cosponsors_for_bill(bill_id, congress=115):
    """
    Get the cosposor data in JSON from a bill ID
    """
    
    req_url = "https://api.propublica.org/congress/v1/{congress}/bills/{bill_id}/cosponsors.json"
    res = requests.get(req_url.format(bill_id=bill_id, congress=congress), headers = HEADERS)
    
    # Extract results from JSON response
    co_sponsors = res.json()['results'][0]['cosponsors']
    
    # Print progress status
    print("Getting cosponsors for bill " + str(bill_id), end="\r")
    
    return co_sponsors

sponsored_bills = active_bills.copy()

# For each bill (row) in the dataframe, return a list of all the cosponsors for that particular bill
# Those results are stored as a new column in the dataframe
sponsored_bills['cosponsors_id'] = sponsored_bills.apply( 
    lambda row: extract_cosponsors(cosponsors_for_bill(row['bill_slug'])) 
    , axis=1
)

# Save the results
sponsored_bills.to_pickle(data_folder + "/bills/bills_and_sponsors.pickle")

### Processing the sponsoring data

This section processes the bill with cosponsors dataframe to store them in a convenient way. The sponsor and cosponsors are merged in a common list and the dataframe is reorganized in a way that each bill is represented by a row and each senator as a column. There is then a one in a particular cell if the corresponding senator sponsored the corresponding bill.

In [ ]:
# Extract and merge sponsors for each bill
bs = pd.read_pickle(data_folder + "bills/bills_and_sponsors.pickle")
bs = bs.apply(lambda row: pd.Series([row['bill_id'], row['cosponsors_id'] + [row['sponsor_id']]], index=['bills','sponsors']), axis=1)
bs = bs.set_index('bills')
bs.head(3)

In [ ]:
# Prepare the one-hot encoding for sponsoring information
df = bs['sponsors'].str.join('|').str.get_dummies()
df = df.transpose()

# Save the results
df.to_pickle(data_folder + "member_matrices/member_to_bill_sponsoring.pickle")

df.head(3)

## Fetching senate members (congress 80 to 115)

Retrieve senate members for each congress and save as a separate file.

In [ ]:
u = "https://api.propublica.org/congress/v1/{congress}/senate/members.json"

for i in range(80, 115 + 1):
    results = requests.get(url = u.format(congress=i), headers=HEADERS)
    df = pd.io.json.json_normalize(results.json()['results'][0]['members'])
    df.to_csv(data_folder + "senate_members/senate_members_{congress}.csv".format(congress=i))
    df.to_pickle(data_folder + "senate_members/senate_members_{congress}.pickle".format(congress=i))

## Fetching Committees from senate

Retrieve all the committees of the senate for congresses 114 and 115

In [ ]:
# Request config
u = "https://api.propublica.org/congress/v1/{congress}/senate/committees.json"

for i in range(114, 115 + 1):
    results_committee = requests.get(url = u.format(congress=i), headers=HEADERS)
    df = pd.io.json.json_normalize(results_committee.json()['results'][0]['committees'])
    df_list = []
    
    # Extract committee information
    for committee_id in df['id']:
        subcommittee_id = df[df['id'] == committee_id]['subcommittees']

        # Fetch data regarding each subcommittee
        for subcommittee in subcommittee_id.values[0]:
            results_sub = requests.get(url = subcommittee['api_uri'], headers=HEADERS)
            if 'results' in results_sub.json():
                df_sub = pd.io.json.json_normalize(results_sub.json()['results'][0]['current_members'])
                df_sub['subcomittee'] = subcommittee['id']
                df_sub['comittee'] = committee_id
                df_list.append(df_sub)

    if len(df_list) > 0:
        df_merged = pd.concat(df_list)
        df_merged.to_csv(data_folder + 'committees/committees_members_{congress}.csv'.format(congress = i))
        df_merged.to_pickle(data_folder + 'committees/committees_members_{congress}.pickle'.format(congress = i))

# Getting voting position by member

Retrieve all the votes casted by each senator. For our project we use votes casted by senators of the 115th congress.

In [4]:
raw_senators = pd.DataFrame()

for i in range (115,116):
    df = pd.read_pickle("data/senate_members/senate_members_{congress}.pickle".format(congress = i))
    df['congress'] = i
    raw_senators = pd.concat([raw_senators, df], sort=False)
    
senators_id = raw_senators['id'].unique()

In [9]:
u = "https://api.propublica.org/congress/v1/members/{member_id}/votes.json?offset={offset}"

import os
import json
import ast

completed_ids = os.listdir("data/votes")

for senator_id in senators_id:

    if "votes_{id}.csv".format(id=senator_id) in completed_ids:
        continue
        
    error_raised = False
    votes_list = []
    data_available = True
    request_offset = 0

    while data_available :
        res = requests.get(url = u.format(member_id = senator_id, offset = request_offset), headers = HEADERS)
        s = re.sub(r'(\s{2,}|\n)', ' ', res.text)

        jObj = json.loads(s)
        
        print(json.dumps(jObj, indent=4))
                
        if(res.status_code == 200):
            data_available = int(jObj['results'][0]['num_results']) > 0

            if data_available:
                # Print progress status
                print(str(senator_id) + " offset: " + str(request_offset), end='\r')
                
                # Extract data from JSON
                votes = jObj['results'][0]['votes']
                votes_list.append(pd.io.json.json_normalize(votes, record_prefix=True))
                
        else:
            print(str(res.status_code))
            error_raised = True
            break
        
        request_offset += 20
        
    if not error_raised and len(votes_list) > 0:
        df = pd.concat(votes_list,sort=False)
        df.to_csv(data_folder + "votes/votes_{id}.csv".format(id=senator_id), index=False)


{
    "status": "OK",
    "copyright": "Copyright (c) 2018 Pro Publica Inc. All Rights Reserved.",
    "results": [
        {
            "member_id": "A000360",
            "num_results": "20",
            "offset": "0",
            "votes": [
                {
                    "member_id": "A000360",
                    "chamber": "Senate",
                    "congress": "115",
                    "session": "2",
                    "roll_call": "274",
                    "vote_uri": "https://api.propublica.org/congress/v1/115/senate/sessions/2/votes/274.json",
                    "bill": {
                        "bill_id": "hr695-115",
                        "number": "H.R.695",
                        "sponsor_id": "S001150",
                        "bill_uri": "https://api.propublica.org/congress/v1/115/bills/hr695.json",
                        "title": "To amend the National Child Protection Act of 1993 to establish a national criminal history background check system and c

{
    "status": "OK",
    "copyright": "Copyright (c) 2018 Pro Publica Inc. All Rights Reserved.",
    "results": [
        {
            "member_id": "A000360",
            "num_results": "20",
            "offset": "20",
            "votes": [
                {
                    "member_id": "A000360",
                    "chamber": "Senate",
                    "congress": "115",
                    "session": "2",
                    "roll_call": "254",
                    "vote_uri": "https://api.propublica.org/congress/v1/115/senate/sessions/2/votes/254.json",
                    "bill": {
                        "bill_id": "pn2556-115",
                        "number": "PN2556",
                        "api_uri": null,
                        "title": null,
                        "latest_action": null
                    },
                    "amendment": {},
                    "nomination": {
                        "nomination_id": "PN2556-115",
                        "

{
    "status": "OK",
    "copyright": "Copyright (c) 2018 Pro Publica Inc. All Rights Reserved.",
    "results": [
        {
            "member_id": "A000360",
            "num_results": "20",
            "offset": "40",
            "votes": [
                {
                    "member_id": "A000360",
                    "chamber": "Senate",
                    "congress": "115",
                    "session": "2",
                    "roll_call": "234",
                    "vote_uri": "https://api.propublica.org/congress/v1/115/senate/sessions/2/votes/234.json",
                    "bill": {
                        "bill_id": "pn1420-115",
                        "number": "PN1420",
                        "api_uri": null,
                        "title": null,
                        "latest_action": null
                    },
                    "amendment": {},
                    "nomination": {
                        "nomination_id": "PN1420-115",
                        "

{
    "status": "OK",
    "copyright": "Copyright (c) 2018 Pro Publica Inc. All Rights Reserved.",
    "results": [
        {
            "member_id": "A000360",
            "num_results": "20",
            "offset": "60",
            "votes": [
                {
                    "member_id": "A000360",
                    "chamber": "Senate",
                    "congress": "115",
                    "session": "2",
                    "roll_call": "214",
                    "vote_uri": "https://api.propublica.org/congress/v1/115/senate/sessions/2/votes/214.json",
                    "bill": {
                        "bill_id": "pn2048-115",
                        "number": "PN2048",
                        "api_uri": null,
                        "title": null,
                        "latest_action": null
                    },
                    "amendment": {},
                    "nomination": {
                        "nomination_id": "PN2048-115",
                        "

{
    "status": "OK",
    "copyright": "Copyright (c) 2018 Pro Publica Inc. All Rights Reserved.",
    "results": [
        {
            "member_id": "A000360",
            "num_results": "20",
            "offset": "80",
            "votes": [
                {
                    "member_id": "A000360",
                    "chamber": "Senate",
                    "congress": "115",
                    "session": "2",
                    "roll_call": "194",
                    "vote_uri": "https://api.propublica.org/congress/v1/115/senate/sessions/2/votes/194.json",
                    "bill": {
                        "bill_id": "pn602-115",
                        "number": "PN602",
                        "api_uri": null,
                        "title": null,
                        "latest_action": null
                    },
                    "amendment": {},
                    "nomination": {
                        "nomination_id": "PN602-115",
                        "num

KeyboardInterrupt: 

# Fetching Lobbying from senate (unused in project)

In [ ]:
re.compile("\((.*)\"([\w ]+)\"(.*)\)")

In [ ]:
# Request config
u = "https://api.propublica.org/congress/v1/lobbying/latest.json?offset={offset}"

votes_list = []
data_available = True
request_offset = 0

while data_available :
    res = requests.get(url = u.format(offset = request_offset), headers = HEADERS)
    j  = res.text.replace("\\\"","").replace("\\","").replace("\")", ")").replace("(\"", "(")

    try:
        res = json.loads(j)       
        if('results' in res):
            #print(request_offset)
            data_available = int(res['results'][0]['num_results']) > 0

            if data_available:
                votes = res['results'][0]['lobbying_representations']
                votes_list.append(pd.io.json.json_normalize(votes, record_prefix=True))
        else:
            print(str(request_offset) + ' - Error: ' + res['error'])
    except:
        print(str(request_offset) + ' - Error: Json File badly encoded')
    request_offset += 20
    
df = pd.concat(votes_list,sort=True)
df.to_csv(data_folder + "lobby/lobby.csv")
df.head()